<h1>Image recognition using  CNN </h1>

<h2>Problem statement :-</h2>

1. Extract image frames from each video and store them in folders “asset1”  and “asset2”

2. Create a third folder “test”

3. MOVE five images from “asset1” and asset2” to the “test” folder.

4. Using the images in “asset1” and “asset2” train an image recognition model that will be able to distinguish images between images in the two folders

5. Apply the model on the ten images stored in “test” folder

In this  notebook I have convert the video files to images by using external source(VLC) which is quite easier.

Both vedio are of 29 frames/second 

**No of Training samples = 34**\
**No of Testing  samples   = 10**

## Importing the libraries

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
% matplotlib inline
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import random
import os
import warnings
warnings.filterwarnings("ignore")

## Importing dataset and image processing

In [0]:
data='ES'
categories = ["asset1","asset2"]
IMG_SIZE=350

In [24]:
training_data = []
from tqdm import tqdm
def create_training_data():
    for category in categories:

        path = os.path.join(data,category) 
        class_num = categories.index(category) 

        for img in tqdm(os.listdir(path)): 
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE)) 
                training_data.append([new_array, class_num]) 
            except Exception as e: 
                pass
            

create_training_data()
print(len(training_data))

100%|██████████| 17/17 [00:00<00:00, 162.23it/s]

34


## Randomly shuffling the data

In [25]:
random.shuffle(training_data)
for sample in training_data:
    print(sample[1])

1
0
0
0
1
1
1
1
0
1
1
0
0
0
1
1
0
1
0
1
0
1
1
0
1
1
0
0
0
1
1
0
0
0


## Taking Training dataset

In [0]:
X = []
y = []
for features,label in training_data:
    X.append(features)
    y.append(label)
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

**Normalizing the training features**

In [0]:
X = X/255.0

## Taking Testing dataset and image processing

In [28]:
testing_data = []
from tqdm import tqdm
paths='ES/test' 

for img in tqdm(os.listdir(paths)):
    img_array = cv2.imread(os.path.join(paths,img) ,cv2.IMREAD_GRAYSCALE)  
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  
    testing_data.append(new_array)  
            
       


100%|██████████| 10/10 [00:00<00:00, 138.53it/s]


In [0]:
X_test = []
for fetrs in testing_data:
    X_test.append(fetrs)
X_test = np.array(X_test).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

**Normalizing** **the** **Testing** **features**

In [0]:
X_test=X_test/255.0

## MODELLING APPLYING CNN

In [0]:
model = Sequential()
model.add(Conv2D(64,(3,3), activation = 'relu', input_shape = X.shape[1:]))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Conv2D(64,(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer="adam",
              loss='binary_crossentropy',
              metrics=['accuracy'])

## Fitting the model to the training dataset

In [32]:
model.fit(X, y, epochs=3, batch_size=2, validation_split=0.1)

Train on 30 samples, validate on 4 samples
Epoch 1/3
30/30 [==============================] - 2s 55ms/sample - loss: 1.1630 - acc: 0.9000 - val_loss: 1.0480e-07 - val_acc: 1.0000
Epoch 2/3
30/30 [==============================] - 1s 43ms/sample - loss: 1.1024e-07 - acc: 1.0000 - val_loss: 3.6264e-07 - val_acc: 1.0000
Epoch 3/3
30/30 [==============================] - 1s 43ms/sample - loss: 6.0963e-05 - acc: 1.0000 - val_loss: 1.0480e-07 - val_acc: 1.0000


## Predicting on Test data

In [0]:
prediction = model.predict(X_test)

In [34]:
prediction.shape

(10, 1)

## Now saving the Testing result on csv file

In [0]:
id_line = []
for p in os.listdir(paths):
        id_line.append(p.split(".")[0])

In [0]:
#prediction are in 2d array we are converting 2d into 1d
pred_value=np.reshape(prediction,(10))

In [0]:
submission_df = pd.DataFrame({'id':id_line, 'label': pred_value })

In [0]:
submission_df.to_csv("Test_result.csv", index=False)